<a href="https://colab.research.google.com/github/Polyakri/projects/blob/main/MachineLearningProjects/RockPaperScissors_Game/4_CNN_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import the necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from itertools import product
from os.path import exists
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2



from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Prepare the data for the CNN

In [2]:
x_data = np.load('drive/MyDrive/MLFinalEx/NoPCA/x_data_full_flipped.npy')
y_data = np.load('drive/MyDrive/MLFinalEx/NoPCA/y_data_full_flipped.npy')

x_data = np.reshape (x_data, (x_data.shape[0],200,300,1))

Resize the data into smaller pictures, it took way too much time to proceed with the 200x300 images.

In [3]:
target_size = (75, 50)

# Create an empty array to store resized images
resized_images = np.empty((x_data.shape[0], target_size[1], target_size[0]))

# Loop through each image and resize
for i in range(x_data.shape[0]):
    img = x_data[i]

    # Resize the image using OpenCV
    resized_img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

    # Store the resized image in the new array
    resized_images[i] = resized_img

print("Resizing complete.")

Resizing complete.


In [4]:
x_data = resized_images

Split into training and test set

In [5]:
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.2, random_state=42, stratify=y_data
)

# Print the shapes of the resulting sets
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

x_train shape: (3500, 50, 75)
x_test shape: (876, 50, 75)
y_train shape: (3500,)
y_test shape: (876,)


#Training the Convolutional Neural Networks

Training all these neural networks will indeed take a lot time so when I found 99.5 % accuracy I stopped

In [6]:
metrics = ['accuracy']
loss = 'binary_crossentropy'
num_classes =3
y_test = tf.one_hot(y_test, depth=num_classes)
y_train = tf.one_hot(y_train, depth=num_classes)
y_train.shape, y_test.shape

(TensorShape([3500, 3]), TensorShape([876, 3]))

In [ ]:

num_conv_layers = [1,2,3]
num_filters = [32, 64]
kernel_size = [(5, 5), (3, 3)]
strides = (1, 1)
mlp_num_hidden_layers = [1, 2, 3]
mlp_units_per_layer = [32, 64, 128, 256, 512]
activation = ['relu', 'sigmoid', 'tanh']
dropout_rate = [0.4]
batch_size = [32]
epochs = [50]
learning_rate = [0.001, 0.01, 0.1]
output_activations = ['softmax', 'sigmoid']

#Put the correct file location
csv_filename = 'drive/MyDrive/MLFinalEx/CNN_full_flipped_results_temp2.csv'

if exists(csv_filename):
    results_df = pd.read_csv(csv_filename)
else:
    results_df = pd.DataFrame(columns=['Num_Conv_Layers', 'Num_Filters', 'Kernel_Size', 'MLP_Num_Hidden_Layers',
                                       'MLP_Units_Per_Layer', 'Activation', 'Dropout_Rate', 'Batch_Size', 'Epochs',
                                       'Learning_Rate', 'Output_Activation', 'Accuracy'])

# Generate all combinations of hyperparameters
param_combinations = product(
    num_conv_layers,
    num_filters,
    kernel_size,
    mlp_num_hidden_layers,
    mlp_units_per_layer,
    activation,
    dropout_rate,
    batch_size,
    epochs,
    learning_rate,
    output_activations
)

for params in param_combinations:
    num_conv, filters, k_size, mlp_layers, mlp_units, act, dropout, b_size, epoch, lr, output_activation = params

    # Check if the combination already exists in the DataFrame
    if results_df[
        (results_df['Num_Conv_Layers'] == num_conv) &
        (results_df['Num_Filters'] == filters) &
        (results_df['Kernel_Size'] == str(k_size)) &  # Convert tuple to string for comparison
        (results_df['MLP_Num_Hidden_Layers'] == mlp_layers) &
        (results_df['MLP_Units_Per_Layer'] == mlp_units) &
        (results_df['Activation'] == act) &
        (results_df['Dropout_Rate'] == dropout) &
        (results_df['Batch_Size'] == b_size) &
        (results_df['Epochs'] == epoch) &
        (results_df['Learning_Rate'] == lr) &
        (results_df['Output_Activation'] == output_activation)
    ].shape[0] > 0:
        print(f"Skipping existing combination: {params}")
        continue

    cnn = tf.keras.Sequential(name='my-cnn')
    cnn.add(tf.keras.layers.Input((50, 75, 1), name='input_layer'))

    for i in range(num_conv):
        cnn.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=k_size, strides=strides, activation=act,
                                       name=f'conv_layer{i + 1}'))
        cnn.add(tf.keras.layers.Dropout(rate=dropout, name=f'dropout_{i + 1}'))
        cnn.add(tf.keras.layers.MaxPooling2D(name=f'maxpool_{i + 1}'))

    # Flattening
    cnn.add(tf.keras.layers.Flatten())

    # MLP
    for i in range(mlp_layers):
        cnn.add(tf.keras.layers.Dense(units=mlp_units, activation=act, name=f'hidden_layer_{i + 1}'))

    cnn.add(tf.keras.layers.Dense(units=num_classes, activation=output_activation, name='output_layer'))

    cnn.summary ()

    cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='categorical_crossentropy',
                metrics=['accuracy'])

    # Assuming x_train, y_train, x_test, y_test are defined
    history = cnn.fit(x_train, y_train, batch_size=b_size, epochs=epoch, validation_data=(x_test, y_test), verbose=1)

    # Get the final accuracy from the training history
    accuracy = history.history['val_accuracy'][-1]

    # Append results to the DataFrame
    results_df = results_df.append({
        'Num_Conv_Layers': num_conv,
        'Num_Filters': filters,
        'Kernel_Size': str(k_size),  # Convert tuple to string for storage
        'MLP_Num_Hidden_Layers': mlp_layers,
        'MLP_Units_Per_Layer': mlp_units,
        'Activation': act,
        'Dropout_Rate': dropout,
        'Batch_Size': b_size,
        'Epochs': epoch,
        'Learning_Rate': lr,
        'Output_Activation': output_activation,
        'Accuracy': accuracy
    }, ignore_index=True)

    # Save the results after every iteration
    results_df.to_csv(csv_filename, index=False)

    # Display the results
    print(f"Hyperparameters: {params}, Accuracy: {accuracy}")
